In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/overview-of-recordings.csv
/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44294866_15891095.wav
/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44263136_25998544.wav
/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44263136_58938609.wav
/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44294866_77416341.wav
/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44323331_53313006.wav
/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44273314_94623228.wav
/

In [ ]:
# %%bash
# cat <<'EOT' > ds_config_zero3.json
# {
#     "fp16": {
#         "enabled": "auto",
#         "loss_scale": 0,
#         "loss_scale_window": 1000,
#         "initial_scale_power": 16,
#         "hysteresis": 2,
#         "min_loss_scale": 1
#     },

#     "optimizer": {
#         "type": "AdamW",
#         "params": {
#             "lr": "auto",
#             "betas": "auto",
#             "eps": "auto",
#             "weight_decay": "auto"
#         }
#     },

#     "scheduler": {
#         "type": "WarmupLR",
#         "params": {
#             "warmup_min_lr": "auto",
#             "warmup_max_lr": "auto",
#             "warmup_num_steps": "auto"
#         }
#     },

#     "zero_optimization": {
#         "stage": 3,
#         "offload_optimizer": {
#             "device": "cpu",
#             "pin_memory": true
#         },
#         "offload_param": {
#             "device": "cpu",
#             "pin_memory": true
#         },
#         "overlap_comm": true,
#         "contiguous_gradients": true,
#         "sub_group_size": 1e9,
#         "reduce_bucket_size": "auto",
#         "stage3_prefetch_bucket_size": "auto",
#         "stage3_param_persistence_threshold": "auto",
#         "stage3_max_live_parameters": 1e9,
#         "stage3_max_reuse_distance": 1e9,
#         "stage3_gather_16bit_weights_on_model_save": true
#     },

#     "gradient_accumulation_steps": "auto",
#     "gradient_clipping": "auto",
#     "steps_per_print": 2000,
#     "train_batch_size": "auto",
#     "train_micro_batch_size_per_gpu": "auto",
#     "wall_clock_breakdown": false
# }
# EOT

In [2]:
df = pd.read_csv("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/overview-of-recordings.csv")

In [3]:
len(df)

6661

In [4]:
df.columns

Index(['audio_clipping', 'audio_clipping:confidence',
       'background_noise_audible', 'background_noise_audible:confidence',
       'overall_quality_of_the_audio', 'quiet_speaker',
       'quiet_speaker:confidence', 'speaker_id', 'file_download', 'file_name',
       'phrase', 'prompt', 'writer_id'],
      dtype='object')

In [5]:
train_audio = os.listdir("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/test")
validate_audio = os.listdir("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate")
test_audio = os.listdir("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train")

In [6]:
len(train_audio), len(validate_audio), len(test_audio)

(5895, 385, 381)

In [7]:
def categories(audio_path):
    if audio_path in train_audio:
        return "train"
    elif audio_path in validate_audio:
        return "validate"
    else:
        return "test"
    

In [8]:
df["category"]=df["file_name"].apply(lambda x: categories(x))

In [9]:
def audio_column(category, file_name):
    if category == "train":
        return os.path.join("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/test",file_name)
    elif category == "validate":
        return os.path.join("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate",file_name)
    elif category == "test":
        return os.path.join("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train",file_name)

In [10]:
df["audio"] = df.apply(lambda x: audio_column(x.category, x.file_name), axis=1)

In [11]:
df.sample(2)

,audio_clipping,audio_clipping:confidence,background_noise_audible,background_noise_audible:confidence,overall_quality_of_the_audio,quiet_speaker,quiet_speaker:confidence,speaker_id,file_download,file_name,phrase,prompt,writer_id,category,audio
3273,no_clipping,1.0,no_noise,1.00,3.00,audible_speaker,1.0,41336418,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_41336418_47832282.wav,My hair is falling out in huge amount,Hair falling out,44118742,train,/kaggle/input/medical-speech-transcription-and...
4249,no_clipping,1.0,no_noise,0.69,3.67,audible_speaker,1.0,34824899,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_34824899_23826302.wav,The pain in my ear is unbearable.,Ear ache,44292353,train,/kaggle/input/medical-speech-transcription-and...


In [12]:
train_df = df[df["category"]=="train"].reset_index(drop=True)
valid_df = df[df["category"]=="validate"].reset_index(drop=True)
test_df = df[df["category"]=="test"].reset_index(drop=True)

In [13]:
len(train_df), len(valid_df), len(test_df)

(5895, 385, 381)

In [14]:
train_df = train_df[["audio","phrase"]]
valid_df = valid_df[["audio","phrase"]]
test_df = test_df[["audio","phrase"]]

In [15]:
from datasets import Audio, DatasetDict, Dataset

In [16]:
data = DatasetDict()

In [17]:
data["train"] = Dataset.from_pandas(train_df)
data["test"] = Dataset.from_pandas(valid_df)

In [18]:
data

DatasetDict({
    train: Dataset({
        features: ['audio', 'phrase'],
        num_rows: 5895
    })
    test: Dataset({
        features: ['audio', 'phrase'],
        num_rows: 385
    })
})

In [19]:
data["train"][0]["audio"]

'/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/test/1249120_43453425_58166571.wav'

In [20]:
import wave
with wave.open(data["train"][0]["audio"], "rb") as wave_file:
    frame_rate = wave_file.getframerate()
    print(frame_rate)

48000


In [21]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="English", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="English", task="transcribe")

In [22]:
sr = feature_extractor.sampling_rate
print(sr)

16000


In [23]:
data = data.cast_column("audio", Audio(sampling_rate=sr))

In [24]:
data

DatasetDict({
    train: Dataset({
        features: ['audio', 'phrase'],
        num_rows: 5895
    })
    test: Dataset({
        features: ['audio', 'phrase'],
        num_rows: 385
    })
})

In [25]:
input_str = data["train"][0]["phrase"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Input:                 When I remember her I feel down
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>When I remember her I feel down<|endoftext|>
Decoded w/out special: When I remember her I feel down
Are equal:             True


In [26]:
data["train"][0]

{'audio': {'path': '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/test/1249120_43453425_58166571.wav',
  'array': array([-0.0031451 , -0.00689955, -0.00757617, ...,  0.03054536,
          0.02678971,  0.0332142 ], dtype=float32),
  'sampling_rate': 16000},
 'phrase': 'When I remember her I feel down'}

In [27]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["phrase"]).input_ids
    return batch


In [30]:
data = data.map(prepare_dataset, remove_columns=data.column_names["train"], num_proc=8)


#0:   0%|          | 0/737 [00:00<?, ?ex/s]

#1:   0%|          | 0/737 [00:00<?, ?ex/s]

#2:   0%|          | 0/737 [00:00<?, ?ex/s]

#3:   0%|          | 0/737 [00:00<?, ?ex/s]

#4:   0%|          | 0/737 [00:00<?, ?ex/s]

...

#4:   0%|          | 0/48 [00:00<?, ?ex/s]

#3:   0%|          | 0/48 [00:00<?, ?ex/s]

#5:   0%|          | 0/48 [00:00<?, ?ex/s]

#6:   0%|          | 0/48 [00:00<?, ?ex/s]

#7:   0%|          | 0/48 [00:00<?, ?ex/s]

In [31]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [32]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [33]:
!pip install evaluate
!pip install jiwer

In [34]:
import evaluate

metric = evaluate.load("wer")


[2023-10-30 11:24:03,113] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [35]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [36]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")


In [37]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


In [ ]:
# !pip install deepspeed

In [ ]:
# import os

# os.environ["MASTER_ADDR"] = "localhost"
# os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
# os.environ["RANK"] = "0"
# os.environ["LOCAL_RANK"] = "0"
# os.environ["WORLD_SIZE"] = "1"

In [38]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/whisper-finetune",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs = 5,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
#     deepspeed="/kaggle/working/ds_config_zero3.json"
)


In [39]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


In [41]:
import torch

torch.cuda.empty_cache()

In [43]:
trainer.train()


wandb: Currently logged in as: shrestha-siddhartha06. Use `wandb login --relogin` to force relogin


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Wer
0,0.383600,0.576342,29.209446
1,0.210100,0.394769,30.236140
2,0.119700,0.302902,27.104723
4,0.052800,0.258305,24.127310
4,0.026100,0.259499,24.050308


TrainOutput(global_step=920, training_loss=0.34437358670908474, metrics={'train_runtime': 8769.2799, 'train_samples_per_second': 3.361, 'train_steps_per_second': 0.105, 'total_flos': 1.90921884696576e+18, 'train_loss': 0.34437358670908474, 'epoch': 4.99})

In [42]:
!nvidia-smi

Mon Oct 30 11:24:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    34W / 250W |   1123MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [44]:
!zip -r file.zip /kaggle/working/whisper-finetune/checkpoint-920

  adding: kaggle/working/whisper-finetune/checkpoint-920/ (stored 0%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/pytorch_model.bin (deflated 8%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/generation_config.json (deflated 72%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/trainer_state.json (deflated 80%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/rng_state.pth (deflated 28%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/config.json (deflated 59%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/optimizer.pt (deflated 7%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/training_args.bin (deflated 49%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/preprocessor_config.json (deflated 42%)
  adding: kaggle/working/whisper-finetune/checkpoint-920/scheduler.pt (deflated 49%)


In [45]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/file.zip')

/kaggle/working/file.zip

In [46]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 7.2 MB/s eta 0:00:0000:0100:01m


In [48]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-j7iel62h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-j7iel62h
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 19.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 50.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 14.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=

In [49]:
import speech_recognition as sr

In [50]:
r = sr.Recognizer()

with sr.AudioFile("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44142156_100535941.wav") as source:
    audio = r.record(source)

In [51]:
transcribe = r.recognize_whisper(audio, language="English",model="/kaggle/working/whisper-finetune/checkpoint-920")

RuntimeError: Model /kaggle/working/whisper-finetune/checkpoint-920 not found; available models = ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large']

In [2]:
from transformers import pipeline


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [22]:
pipe = pipeline("automatic-speech-recognition",model="hiiamsid/whisper-finetune", device="cuda")  # change to "your-username/the-name-you-picked"

In [14]:
file_path = "/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train"
aud = os.listdir(file_path)

In [15]:
a = aud[:7]

In [17]:
a = [os.path.join(file_path,i) for i in a]

In [18]:
a

['/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44160489_107692984.wav',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44176037_39613511.wav',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44176037_85065458.wav',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44194084_23344116.wav',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44220382_48462181.wav',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44160489_93769999.wav',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/12

In [19]:
import time

In [23]:
def get_result_from_finetuned_model(aud):
    start = time.perf_counter()
    txt =  pipe(aud)["text"]
    print(txt)
    print("Time taken: ", time.perf_counter()-start)

In [24]:
for i in a:
    get_result_from_finetuned_model(i)

My coach is very heavy and I have mocchus
Time taken:  5.744341218000045
 It is like I have a needle pushing through my heart
Time taken:  0.3637579220001044
 Every time I comb my hair there are so many hair in my comb
Time taken:  0.393166046000033
 I am always cold, even when I am wearing layers.
Time taken:  0.3801703159999761
 I chopped off the tap of my finger while I was cutting some cardboard and I cannot stop the bleeding.
Time taken:  0.4504301829999804
 It started as a teminitum but today the pain is throwing an unbearable
Time taken:  0.44529561199988166
There is a tingling sensation in my neck
Time taken:  0.3380317109999851


In [25]:
pipe1 = pipeline("automatic-speech-recognition",model="openai/whisper-base", device="cuda")  # change to "your-username/the-name-you-picked"

In [26]:
def get_result_from_finetuned_model(aud):
    start = time.perf_counter()
    txt =  pipe1(aud)["text"]
    print(txt)
    print("Time taken: ", time.perf_counter()-start)

In [27]:
for i in a:
    get_result_from_finetuned_model(i)

 My coach is very heavy and I have moccos.
Time taken:  0.45818368699997336
 It is like I have a needle pushing through my heart.
Time taken:  0.35851314699993964
 Every time I comb my hair, there are so many hair in my comb.
Time taken:  0.3765148620000218
 I am always cold, even when I am wearing layers.
Time taken:  0.3758162970000285
 I chopped off the tip of my finger while I was cutting some cardboard and I cannot stop the bleeding.
Time taken:  0.41341507399988586
 It started as a timid was. But today the pain, throwing and unbearable.
Time taken:  0.4316059369998584
 There is a tingling sensation in my life.
Time taken:  0.34117940000010094


In [28]:
pipe1 = pipeline("automatic-speech-recognition",model="openai/whisper-large", device="cuda")  # change to "your-username/the-name-you-picked"

In [29]:
def get_result_from_finetuned_model(aud):
    start = time.perf_counter()
    txt =  pipe1(aud)["text"]
    print(txt)
    print("Time taken: ", time.perf_counter()-start)

In [30]:
for i in a:
    get_result_from_finetuned_model(i)

 My couch is very heavy and I have mucous.
Time taken:  1.1415414800001145
 It is like I have a needle pushing through my heart.
Time taken:  1.0757834699998057
 Every time I comb my hair, there are so many hairs in my comb.
Time taken:  1.2070507990001715
 I am always cold even when I am wearing layers.
Time taken:  1.071062768000047
 I chopped off the tip of my finger while I was cutting some cardboard and I cannot stop the bleeding.
Time taken:  1.3629286190000585
 it stuttered as a timid it was but today the pain, throbbing and unbearable
Time taken:  1.443661044999999
 There is a tingling sensation in my neck.
Time taken:  1.0286061729998437


In [31]:
aaa = [os.path.join(file_path,i) for i in aud]

In [32]:
dic = {}

def get_result_from_finetuned_model(aud):
    start = time.perf_counter()
    txt =  pipe1(aud)["text"]
#     print(txt)
    print("Time taken: ", time.perf_counter()-start)
    dic[aud] = txt

In [33]:
len(aaa)

381

In [34]:
for i in aaa:
    get_result_from_finetuned_model(i)

Time taken:  1.1329236100000344
Time taken:  1.1297575430000961
Time taken:  1.2042098980000446


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Time taken:  1.0734948260001147


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Time taken:  1.355039073999933


...

Time taken:  1.2471258059999855


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Time taken:  1.12889687400002


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Time taken:  1.3108298399997693


In [36]:
dic

{'/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44160489_107692984.wav': ' My couch is very heavy and I have mucous.',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44176037_39613511.wav': ' It is like I have a needle pushing through my heart.',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44176037_85065458.wav': ' Every time I comb my hair, there are so many hairs in my comb.',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44194084_23344116.wav': ' I am always cold even when I am wearing layers.',
 '/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44220382_48462181.wav': ' I chopped off the tip of my finger while

In [63]:
pipe("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44142156_100535941.wav")["text"]

'I have some terrible problems when i am breathing during the night.'

In [65]:
pipe1("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44142156_100535941.wav")["text"]

' I have some terrible problems when I am breathing during the night.'

In [54]:
kwargs = {
    "language": "en",
    "model_name": "Whisper Base Medical",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-base",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [58]:
trainer.push_to_hub(**kwargs)

training_args.bin:   0%|          | 0.00/4.22k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/290M [00:00<?, ?B/s]

'https://huggingface.co/hiiamsid/whisper-finetune/tree/main/'

In [61]:
tokenizer.push_to_hub("hiiamsid/whisper-finetune")

CommitInfo(commit_url='https://huggingface.co/hiiamsid/whisper-finetune/commit/13ed92ec135d7fd6350a2d1db25701acf7cbb5cf', commit_message='Upload tokenizer', commit_description='', oid='13ed92ec135d7fd6350a2d1db25701acf7cbb5cf', pr_url=None, pr_revision=None, pr_num=None)

In [56]:
from huggingface_hub import notebook_login

In [57]:
notebook_login()

In [66]:
test_df.head()

,audio,phrase
0,/kaggle/input/medical-speech-transcription-and...,I have a sharp pain in my lower stomach.
1,/kaggle/input/medical-speech-transcription-and...,Don't cry
2,/kaggle/input/medical-speech-transcription-and...,When I get out of bed in the morning my body f...
3,/kaggle/input/medical-speech-transcription-and...,i have a great pain in my thorax from heart in...
4,/kaggle/input/medical-speech-transcription-and...,I have a hard muscle pain since i went to the gym


In [70]:
def get_result_from_finetuned_model(aud):
    print("*"*15)
    return pipe(aud)["text"]

In [71]:
def get_result_from_non_finetuned_model(aud):
    print("*"*15)
    return pipe1(aud)["text"]

In [72]:
test_df["non_finetuned"] = test_df["audio"].apply(lambda x: get_result_from_non_finetuned_model(x))
test_df["finetuned"] = test_df["audio"].apply(lambda x: get_result_from_finetuned_model(x))


***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
***************
********

In [73]:
test_df.head()

,audio,phrase,non_finetuned,finetuned
0,/kaggle/input/medical-speech-transcription-and...,I have a sharp pain in my lower stomach.,I have a sharp pain in my lower,i have a sharp pain in my lower stomach
1,/kaggle/input/medical-speech-transcription-and...,Don't cry,Don't cry.,don't cry
2,/kaggle/input/medical-speech-transcription-and...,When I get out of bed in the morning my body f...,When I got out of bed in the morning my body ...,When I get out of bed in the morning my body ...
3,/kaggle/input/medical-speech-transcription-and...,i have a great pain in my thorax from heart in...,I have a great pain in my thorax from heart i...,I have a great pain in my thorax from heart i...
4,/kaggle/input/medical-speech-transcription-and...,I have a hard muscle pain since i went to the gym,I have a hard muscle pain since I went to the...,I have a hard muscle pain since I went to the...


In [74]:
ls

ds_config_zero3.json  file.zip  wandb/  whisper-finetune/


In [75]:
test_df.to_csv("res.csv", index=None)